# Random_Forests

In [2]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 68kB/s 
     |████████████████████████████████| 204kB 51.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=a36fcbebd504686f17c10086bf5f7abc449182eb2711beade98e7bff6715483a
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('random_forest').getOrCreate()

In [4]:
df=spark.read.csv('/content/drive/MyDrive/datacamp/affairs.csv',inferSchema=True,header=True)

In [5]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)



In [6]:
df.show(5)

+-------------+----+-----------+--------+---------+-------+
|rate_marriage| age|yrs_married|children|religious|affairs|
+-------------+----+-----------+--------+---------+-------+
|            5|32.0|        6.0|     1.0|        3|      0|
|            4|22.0|        2.5|     0.0|        2|      0|
|            3|32.0|        9.0|     3.0|        3|      1|
|            3|27.0|       13.0|     3.0|        1|      1|
|            4|22.0|        2.5|     0.0|        1|      1|
+-------------+----+-----------+--------+---------+-------+
only showing top 5 rows



In [7]:
df.describe().select('summary','rate_marriage','age','yrs_married','children','religious').show()

+-------+------------------+------------------+-----------------+------------------+------------------+
|summary|     rate_marriage|               age|      yrs_married|          children|         religious|
+-------+------------------+------------------+-----------------+------------------+------------------+
|  count|              6366|              6366|             6366|              6366|              6366|
|   mean| 4.109644989004084|29.082862079798932| 9.00942507068803|1.3968740182218033|2.4261702796104303|
| stddev|0.9614295945655025| 6.847881883668817|7.280119972766412| 1.433470828560344|0.8783688402641785|
|    min|                 1|              17.5|              0.5|               0.0|                 1|
|    max|                 5|              42.0|             23.0|               5.5|                 4|
+-------+------------------+------------------+-----------------+------------------+------------------+



In [9]:
df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 2053|
|      0| 4313|
+-------+-----+



In [10]:
df.groupBy('rate_marriage').count().show()

+-------------+-----+
|rate_marriage|count|
+-------------+-----+
|            1|   99|
|            3|  993|
|            5| 2684|
|            4| 2242|
|            2|  348|
+-------------+-----+



In [11]:
df.groupBy('rate_marriage','affairs').count().orderBy('rate_marriage','affairs','count',ascending=True).show()

+-------------+-------+-----+
|rate_marriage|affairs|count|
+-------------+-------+-----+
|            1|      0|   25|
|            1|      1|   74|
|            2|      0|  127|
|            2|      1|  221|
|            3|      0|  446|
|            3|      1|  547|
|            4|      0| 1518|
|            4|      1|  724|
|            5|      0| 2197|
|            5|      1|  487|
+-------------+-------+-----+



In [12]:
df.groupBy('religious','affairs').count().orderBy('religious','affairs','count',ascending=True).show()

+---------+-------+-----+
|religious|affairs|count|
+---------+-------+-----+
|        1|      0|  613|
|        1|      1|  408|
|        2|      0| 1448|
|        2|      1|  819|
|        3|      0| 1715|
|        3|      1|  707|
|        4|      0|  537|
|        4|      1|  119|
+---------+-------+-----+



In [13]:
df.groupBy('children','affairs').count().orderBy('children','affairs','count',ascending=True).show()

+--------+-------+-----+
|children|affairs|count|
+--------+-------+-----+
|     0.0|      0| 1912|
|     0.0|      1|  502|
|     1.0|      0|  747|
|     1.0|      1|  412|
|     2.0|      0|  873|
|     2.0|      1|  608|
|     3.0|      0|  460|
|     3.0|      1|  321|
|     4.0|      0|  197|
|     4.0|      1|  131|
|     5.5|      0|  124|
|     5.5|      1|   79|
+--------+-------+-----+



In [14]:
df.groupBy('affairs').mean().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------+
|affairs|avg(rate_marriage)|          avg(age)|  avg(yrs_married)|     avg(children)|    avg(religious)|avg(affairs)|
+-------+------------------+------------------+------------------+------------------+------------------+------------+
|      1|3.6473453482708234|30.537018996590355|11.152459814905017|1.7289332683877252| 2.261568436434486|         1.0|
|      0| 4.329700904242986| 28.39067934152562| 7.989334569904939|1.2388128912589844|2.5045212149316023|         0.0|
+-------+------------------+------------------+------------------+------------------+------------------+------------+



In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
df_assembler = VectorAssembler(inputCols=['rate_marriage', 'age', 'yrs_married', 'children', 'religious'], outputCol="features")
df = df_assembler.transform(df)

In [17]:
df.printSchema()

root
 |-- rate_marriage: integer (nullable = true)
 |-- age: double (nullable = true)
 |-- yrs_married: double (nullable = true)
 |-- children: double (nullable = true)
 |-- religious: integer (nullable = true)
 |-- affairs: integer (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
df.select(['features','affairs']).show(10,False)

+-----------------------+-------+
|features               |affairs|
+-----------------------+-------+
|[5.0,32.0,6.0,1.0,3.0] |0      |
|[4.0,22.0,2.5,0.0,2.0] |0      |
|[3.0,32.0,9.0,3.0,3.0] |1      |
|[3.0,27.0,13.0,3.0,1.0]|1      |
|[4.0,22.0,2.5,0.0,1.0] |1      |
|[4.0,37.0,16.5,4.0,3.0]|1      |
|[5.0,27.0,9.0,1.0,1.0] |1      |
|[4.0,27.0,9.0,0.0,2.0] |1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
|[5.0,37.0,23.0,5.5,2.0]|1      |
+-----------------------+-------+
only showing top 10 rows



In [19]:
model_df=df.select(['features','affairs'])

In [20]:
train_df,test_df=model_df.randomSplit([0.75,0.25])

In [21]:
train_df.count()

4774

In [22]:
train_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1| 1527|
|      0| 3247|
+-------+-----+



In [23]:
test_df.groupBy('affairs').count().show()

+-------+-----+
|affairs|count|
+-------+-----+
|      1|  526|
|      0| 1066|
+-------+-----+



In [24]:
from pyspark.ml.classification import RandomForestClassifier

In [25]:
rf_classifier=RandomForestClassifier(labelCol='affairs',numTrees=50).fit(train_df)

In [26]:
rf_predictions=rf_classifier.transform(test_df)

In [27]:
rf_predictions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[33.9359343958634...|[0.67871868791726...|       0.0|
|[1.0,22.0,2.5,1.0...|      1|[17.8787459702051...|[0.35757491940410...|       1.0|
|[1.0,22.0,2.5,1.0...|      0|[20.0482731866041...|[0.40096546373208...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[20.0482731866041...|[0.40096546373208...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[16.1452566777076...|[0.32290513355415...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[17.4474161254095...|[0.34894832250819...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[16.2659123956467...|[0.32531824791293...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.1166239645535...|[0.34233247929107...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[15.8871175691767...|[0.31774235138353...|    

In [28]:
rf_predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0| 1322|
|       1.0|  270|
+----------+-----+



In [29]:
rf_predictions.select(['probability','affairs','prediction']).show(10,False)

+----------------------------------------+-------+----------+
|probability                             |affairs|prediction|
+----------------------------------------+-------+----------+
|[0.6787186879172686,0.32128131208273125]|0      |0.0       |
|[0.35757491940410324,0.6424250805958969]|1      |1.0       |
|[0.40096546373208236,0.5990345362679177]|0      |1.0       |
|[0.40096546373208236,0.5990345362679177]|1      |1.0       |
|[0.32290513355415223,0.6770948664458477]|0      |1.0       |
|[0.3489483225081903,0.6510516774918096] |1      |1.0       |
|[0.3253182479129345,0.6746817520870654] |0      |1.0       |
|[0.3423324792910714,0.6576675207089286] |0      |1.0       |
|[0.3177423513835357,0.6822576486164643] |1      |1.0       |
|[0.33246931988904377,0.6675306801109562]|1      |1.0       |
+----------------------------------------+-------+----------+
only showing top 10 rows



In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='affairs',metricName='accuracy').evaluate(rf_predictions)

In [32]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 70%


In [33]:
print(rf_accuracy)

0.7022613065326633


In [34]:
rf_precision=MulticlassClassificationEvaluator(labelCol='affairs',metricName='weightedPrecision').evaluate(rf_predictions)

In [35]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 68%


In [36]:
rf_precision

0.6817416071222927

In [37]:
rf_auc=BinaryClassificationEvaluator(labelCol='affairs').evaluate(rf_predictions)
print(rf_auc)

0.7247599497784976


In [38]:
rf_classifier.featureImportances

SparseVector(5, {0: 0.6559, 1: 0.0222, 2: 0.2091, 3: 0.0594, 4: 0.0534})

In [39]:
df.schema["features"].metadata["ml_attr"]["attrs"]

{'numeric': [{'idx': 0, 'name': 'rate_marriage'},
  {'idx': 1, 'name': 'age'},
  {'idx': 2, 'name': 'yrs_married'},
  {'idx': 3, 'name': 'children'},
  {'idx': 4, 'name': 'religious'}]}

In [40]:
pwd

'/content'

In [41]:
rf_classifier.save("/content/drive/MyDrive/datacamp/RF_model")

In [42]:
from pyspark.ml.classification import RandomForestClassificationModel

In [43]:
rf=RandomForestClassificationModel.load("/content/drive/MyDrive/datacamp/RF_model")

In [44]:
model_preditions=rf.transform(test_df)

In [45]:
model_preditions.show()

+--------------------+-------+--------------------+--------------------+----------+
|            features|affairs|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[1.0,17.5,0.5,0.0...|      0|[33.9359343958634...|[0.67871868791726...|       0.0|
|[1.0,22.0,2.5,1.0...|      1|[17.8787459702051...|[0.35757491940410...|       1.0|
|[1.0,22.0,2.5,1.0...|      0|[20.0482731866041...|[0.40096546373208...|       1.0|
|[1.0,22.0,2.5,1.0...|      1|[20.0482731866041...|[0.40096546373208...|       1.0|
|[1.0,27.0,2.5,0.0...|      0|[16.1452566777076...|[0.32290513355415...|       1.0|
|[1.0,27.0,2.5,0.0...|      1|[17.4474161254095...|[0.34894832250819...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[16.2659123956467...|[0.32531824791293...|       1.0|
|[1.0,27.0,6.0,1.0...|      0|[17.1166239645535...|[0.34233247929107...|       1.0|
|[1.0,27.0,6.0,2.0...|      1|[15.8871175691767...|[0.31774235138353...|    